In [1]:
DATA_PATH = "survey_results_public.csv"

# Importing Data

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows=1000

In [3]:
raw_df = pd.read_csv(DATA_PATH)

# Data Exploration 

In [4]:
raw_df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [5]:
raw_df.shape

(64461, 61)

In [6]:
raw_df.iloc[0]

Respondent                                                                      1
MainBranch                                         I am a developer by profession
Hobbyist                                                                      Yes
Age                                                                           NaN
Age1stCode                                                                     13
CompFreq                                                                  Monthly
CompTotal                                                                     NaN
ConvertedComp                                                                 NaN
Country                                                                   Germany
CurrencyDesc                                                        European Euro
CurrencySymbol                                                                EUR
DatabaseDesireNextYear                                       Microsoft SQL Server
DatabaseWorkedWi

In [7]:
raw_df.describe()

,Respondent,Age,CompTotal,ConvertedComp,WorkWeekHrs
count,64461.000000,45446.000000,3.482600e+04,3.475600e+04,41151.000000
mean,32554.079738,30.834111,3.190464e+242,1.037561e+05,40.782174
std,18967.442360,9.585392,inf,2.268853e+05,17.816383
min,1.000000,1.000000,0.000000e+00,0.000000e+00,1.000000
25%,16116.000000,24.000000,2.000000e+04,2.464800e+04,40.000000
50%,32231.000000,29.000000,6.300000e+04,5.404900e+04,40.000000
75%,49142.000000,35.000000,1.250000e+05,9.500000e+04,44.000000
max,65639.000000,279.000000,1.111111e+247,2.000000e+06,475.000000


In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Respondent                    64461 non-null  int64  
 1   MainBranch                    64162 non-null  object 
 2   Hobbyist                      64416 non-null  object 
 3   Age                           45446 non-null  float64
 4   Age1stCode                    57900 non-null  object 
 5   CompFreq                      40069 non-null  object 
 6   CompTotal                     34826 non-null  float64
 7   ConvertedComp                 34756 non-null  float64
 8   Country                       64072 non-null  object 
 9   CurrencyDesc                  45472 non-null  object 
 10  CurrencySymbol                45472 non-null  object 
 11  DatabaseDesireNextYear        44070 non-null  object 
 12  DatabaseWorkedWith            49537 non-null  object 
 13  D

In [9]:
raw_df.isnull().sum()

Respondent                          0
MainBranch                        299
Hobbyist                           45
Age                             19015
Age1stCode                       6561
CompFreq                        24392
CompTotal                       29635
ConvertedComp                   29705
Country                           389
CurrencyDesc                    18989
CurrencySymbol                  18989
DatabaseDesireNextYear          20391
DatabaseWorkedWith              14924
DevType                         15091
EdLevel                          7030
Employment                        607
Ethnicity                       18513
Gender                          13904
JobFactors                      15112
JobSat                          19267
JobSeek                         12734
LanguageDesireNextYear          10348
LanguageWorkedWith               7083
MiscTechDesireNextYear          22082
MiscTechWorkedWith              24147
NEWCollabToolsDesireNextYear    17174
NEWCollabToo

It seems that our data need some manipulation before analysis.

# Data Preprocessing

# Helper Functions

In [10]:
def split_answers(data_series, delimiter=";"):
    """ 
    Split multiple answers in a single string 
    to a list of single strings each represnting a single answers 

    Parameters:
    * data_series (pd.Series): String series with answers 
    * delimiter (string): Another decimal integer 
    Defaults to ";"
    
    Returns: (pd.Series): If column contains 
    """
    # Sub functions 
    def is_splittable(pd_series, delimiter):
        """ Check if results multiple should be splitted - Returns boolean """    
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter): 
        """Function to split single answer"""
        return pd_series.str.split(delimiter)
    
    # Check if multiple answers exist - if none: return original 
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any():
        return data_series
    
    # Else, split each value to a list 
    modified_series = split_answer(data_series, delimiter)    
    
    # Replace NAs with empty lists 
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    return modified_series

**1. Split Multiple Answers**

In [11]:
object_cols = raw_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    raw_df[col] = split_answers(raw_df[col])

In [12]:
raw_df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",[ASP.NET Core],"[ASP.NET, ASP.NET Core]",Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",[],[],Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,[],[],Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",[],[],Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...","[Django, Ruby on Rails]",[Ruby on Rails],Just as welcome now as I felt last year,NaN,15,8


**2. Replace Values and parse**
<br>
Those Columns should'nt be type object , instead float

In [13]:
rep = ['YearsCode' , 'YearsCodePro']
for c in rep :
    print(c)
    print(raw_df[c].unique())
    print("*********")

YearsCode
['36' '7' '4' '15' '6' '17' '8' '10' '35' '5' '37' '19' '9' '22' '30' '23'
 '20' '2' 'Less than 1 year' '3' '13' '25' '16' '43' '11' '38' '33' nan
 '24' '21' '12' '40' '27' '50' '46' '14' '18' '28' '32' '44' '26' '42'
 '31' '34' '29' '1' '39' '41' '45' 'More than 50 years' '47' '49' '48']
*********
YearsCodePro
['27' '4' nan '8' '13' '2' '7' '20' '1' '23' '3' '12' '17' '18' '10' '14'
 '29' '6' '28' '9' '15' '11' '16' '25' 'Less than 1 year' '5' '21' '19'
 '35' '24' '32' '22' '30' '38' '26' '40' '33' '31' 'More than 50 years'
 '34' '36' '39' '37' '41' '45' '47' '42' '46' '50' '43' '44' '48' '49']
*********


In [14]:
raw_df['YearsCode'].replace('Less than 1 year' ,'0' , inplace=True)
raw_df['YearsCode'].replace('More than 50 years' ,'51' , inplace=True)
raw_df['YearsCode'] = pd.to_numeric(raw_df['YearsCode'])

In [15]:
raw_df['YearsCodePro'].replace('Less than 1 year' ,'0' , inplace=True)
raw_df['YearsCodePro'].replace('More than 50 years' ,'51' , inplace=True)
raw_df['YearsCodePro'] = pd.to_numeric(raw_df['YearsCodePro'])

**3. Columns Selection**
<br>
We'll Select the Columns We Need According to Our Business Case

In [16]:
raw_df.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedComp', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseDesireNextYear', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'MiscTechDesireNextYear', 'MiscTechWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformDesireNextYear',
       'PlatformWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'WebframeDesireNextYear',
  

In [17]:
needed_cols = ['DevType','LanguageWorkedWith',
                  'DatabaseWorkedWith',
                  'WebframeWorkedWith',
                  'MiscTechWorkedWith',]

In [18]:
raw_df = raw_df[needed_cols]

In [19]:
raw_df.head()

,DevType,LanguageWorkedWith,DatabaseWorkedWith,WebframeWorkedWith,MiscTechWorkedWith
0,"[Developer, desktop or enterprise applications...","[C#, HTML/CSS, JavaScript]","[Elasticsearch, Microsoft SQL Server, Oracle]","[ASP.NET, ASP.NET Core]","[.NET, .NET Core]"
1,"[Developer, full-stack, Developer, mobile]","[JavaScript, Swift]",[],[],[React Native]
2,[],"[Objective-C, Python, Swift]",[],[],[]
3,[],[],[],[],[]
4,[],"[HTML/CSS, Ruby, SQL]","[MySQL, PostgreSQL, Redis, SQLite]",[Ruby on Rails],[Ansible]


# Performing OneHotEncoding

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer
df = raw_df.copy()
encoded_dfs = {}
for col in needed_cols:
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                               columns=binarizer.classes_,
                               index=df[col].index)
    encoded_dfs[col] = encoded_df
df = pd.concat(encoded_dfs, axis=1)

In [21]:
df

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
1                       0                        0   
2                       0                        0   
3                       0                        0   
4                       0                        0   
...                   ...                      ...   
64456                   0                        0   
64457                   0                        0   
64458                   0                        0   
64459                   0                        0   
64460                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
1                                                 0                      0   
2                                                 0                      0   
3                                                 0                      0   
4                                                 0                      0   
...                                             ...                    ...   
64456                                             0                      0   
64457                                             0                      0   
64458                                             0                      0   
64459                                             0                      0   
64460                                             0                      0   

                                                                            \
      Designer DevOps specialist Developer, QA or test Developer, back-end   
0            0                 0                     0                   0   
1            0                 0                     0                   0   
2            0                 0                     0                   0   
3            0                 0                     0                   0   
4            0                 0                     0                   0   
...        ...               ...                   ...                 ...   
64456        0                 0                     0                   0   
64457        0                 0                     0                   0   
64458        0                 0                     0                   0   
64459        0                 0                     0                   0   
64460        0                 0                     0                   0   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 1   
1                                                 0   
2                                                 0   
3                                                 0   
4                                                 0   
...                                             ...   
64456                                             0   
64457                                             0   
64458                                             0   
64459                                             0   
64460                                             0   

                                                   ... MiscTechWorkedWith  \
      Developer, embedded applications or devices  ...            Node.js   
0                                               0  ...                  0   
1                                               0  ...                  0   
2                                               0  ...                  0   
3                                               0  ...                  0   
4                                               0  ...                  0   
...                           

# Saving Data For Next Notebook

In [69]:
import os

df_path = os.path.join(r"E:\", "df.pkl")
df.to_pickle(df_path)

raw_df_path = os.path.join(r"E:\", "processed_data.pkl")
raw_df.to_pickle(raw_df_path)